In [7]:
import Bio
from Bio import Entrez, SeqIO, Seq

In [8]:
Entrez.email = "joshsteinbecker@gmail.com"

In [9]:
handle = Entrez.efetch(db="nucleotide", id="NC_006577.2", rettype="gb", retmode="text") #IOWRAPPER
result = handle.readlines() #List of strings
result

['LOCUS       NC_006577              29926 bp ss-RNA     linear   VRL 20-NOV-2020\n',
 'DEFINITION  Human coronavirus HKU1, complete genome.\n',
 'ACCESSION   NC_006577\n',
 'VERSION     NC_006577.2\n',
 'DBLINK      BioProject: PRJNA485481\n',
 'KEYWORDS    RefSeq.\n',
 'SOURCE      Human coronavirus HKU1 (HCoV-HKU1)\n',
 '  ORGANISM  Human coronavirus HKU1\n',
 '            Viruses; Riboviria; Orthornavirae; Pisuviricota; Pisoniviricetes;\n',
 '            Nidovirales; Cornidovirineae; Coronaviridae; Orthocoronavirinae;\n',
 '            Betacoronavirus; Embecovirus.\n',
 'REFERENCE   1  (bases 1 to 29926)\n',
 '  AUTHORS   Woo,P.C., Lau,S.K., Chu,C.M., Chan,K.H., Tsoi,H.W., Huang,Y.,\n',
 '            Wong,B.H., Poon,R.W., Cai,J.J., Luk,W.K., Poon,L.L., Wong,S.S.,\n',
 '            Guan,Y., Peiris,J.S. and Yuen,K.Y.\n',
 '  TITLE     Characterization and complete genome sequence of a novel\n',
 '            coronavirus, coronavirus HKU1, from patients with pneumonia\n',
 '  JOURNAL 